# Financial Fraud Detection

- The objective of this notebook is to showcase the usage of the [___financial-fraud-training___ container](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/cugraph/containers/financial-fraud-training) and how to deploy the produced trained models on [NVIDIA Dynamo-Triton](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/tritonserver).
- We use [IBM TabFormer](https://github.com/IBM/TabFormer) as an example dataset and the dataset is preprocess before model training

NOTE:
* The preprocessing code is written specifically for the TabFormer dataset and will not work with other datasets.
* Additionally, a familiarity with [Jupyter](https://docs.jupyter.org/en/latest/what_is_jupyter.html) is assumed.

# Environment Setup (Sagemaker studio)
This Notebook is designed to work in a Sagemaker studio jupyter lab notebook

Please create a Conda environment and add that to the notebook - See the [README - Setup Development Environment section](../README.md) file

In [ ]:
!nvidia-smi --version

### Activate the env in the kernel

Now choose the `fraud_blueprint_env` kernel from within jupyterlab for this notebook

-----
## Import libraries

In [ ]:
import os
import sys
import json
import time
import boto3

## Prepare the AWS Environment variables from the infrastructure stack

In [ ]:
# Add the "src" directory to the search path
src_dir = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), 'src'))
sys.path.insert(0, src_dir) 

In [ ]:
from utils.aws import get_cfn_output, get_inference_host

In [ ]:
ssm_client = boto3.client('ssm')
cfn_client = boto3.client('cloudformation')
elb_client = boto3.client('elbv2', region_name="us-east-1")

bucket_name = get_cfn_output("NvidiaFraudDetectionBlueprintModelExtractor", "SourceBucketName")

sagemaker_training_role = get_cfn_output("NvidiaFraudDetectionTrainingRole", "SageMakerRoleArn")

training_repo = get_cfn_output("NvidiaFraudDetectionTrainingImageRepo", "TrainingImageRepoUri")

inference_host = get_inference_host()

print(f"Bucket Name: {bucket_name}")
print(f"Training Role: {sagemaker_training_role}")
print(f"Training Repo: {training_repo}")
print(f"Inference Host: {inference_host}")

----
# Step 1: Get and Prepare the data

## For Local
1. Download the dataset: https://ibm.ent.box.com/v/tabformer-data/folder/130747715605
2. untar and uncompreess the file: `tar -xvzf ./transactions.tgz`
3. Put card_transaction.v1.csv in in the `data/TabFormer/raw` folder


## For Brev 
1. Download the dataset: https://ibm.ent.box.com/v/tabformer-data/folder/130747715605
2. In the Jupyter notebook window, use the "File Browser" section to the data/Tabformer/raw folder
3. Drag-and-drop the "transactions.tgz" file into the folder
    - There is also an "upload" option that displays a file selector
    - Please wait for the upload to finish, it could take a while, by lookign at the status indocator at the bottom of the window
4. Now uncompress and untar by running the following command
    - Note: if somethign goes wrong you will need to delete the file rather than trying to overwrite it.

In [ ]:
# verify that the compressed file was uploaded successfully - the size should be 266M
!ls -lh ../data/TabFormer/raw

In [ ]:
# Uncompress/untar the file
!tar xvzf ../data/TabFormer/raw/transactions.tgz -C ../data/TabFormer/raw/

__If__ drag-and-drop is not working, please run the [Download TabFormer](./extra/download-tabformer.ipynb) notebook is the "extra" folder 

## Check data folder structure
The goal is to produce the following structure

```
.
    data
    └── TabFormer
        └── raw
            └── card_transaction.v1.csv
```

In [ ]:
# Once the raw data is placed as described above, set the path to the TabFormer directory

# Change this path to point to TabFormer data
data_root_dir = os.path.abspath('../data/TabFormer/') 
# Change this path to the directory where you want to save your model
model_output_dir = os.path.join(data_root_dir, 'trained_models')

# Path to save the trained model
os.makedirs(model_output_dir, exist_ok=True)

### Define python function to print directory tree

In [ ]:
from utils.plotting import print_tree

In [ ]:
# Check if the raw data has been placed properly
print_tree(data_root_dir)

---
# Step 2: Preprocess the data 
- Import the Python function for preprocessing the TabFormer data
- Call `preprocess_TabFormer` function to prepare the data

NOTE: The preprocessing can takes a few minutes


In [ ]:
from data_preprocessing.raw_data_processing import load_and_clean_tabformer
from data_preprocessing.xgboost_data_generation import generate_xgboost_features
from data_preprocessing.gnn_data_generation import generate_gnn_graph_data

bundle = load_and_clean_tabformer(
    base_path=data_root_dir,              # Folder containing 'raw/card_transaction.v1.csv'
    csv_name="card_transaction.v1.csv"
)

xgb_transformer, columns_of_transformed_data = generate_xgboost_features(
    cleaned_data_bundle=bundle,
    output_dir=os.path.join(data_root_dir, "xgb"),
    data_split_year=2018)

user_mask_map, mx_mask_map, tx_mask_map = generate_gnn_graph_data(
    cleaned_data_bundle=bundle, 
    output_dir=os.path.join(data_root_dir, "gnn"),
    data_split_year=2018, 
    xgb_transformer=xgb_transformer, 
    columns_of_transformed_txs=columns_of_transformed_data)

In [ ]:
print_tree(data_root_dir)

### Graph Summary
Summarize the bipartite graph structure created by preprocessing.

In [ ]:
from utils.plotting import summarize_graph
# Run the summary
gnn_dir = os.path.join(data_root_dir, "gnn")
graph_stats = summarize_graph(gnn_dir)

In [ ]:
# copy data to S3 to get pulled during training
! aws s3 sync "../data/TabFormer/" s3://$bucket_name/data/ --exclude "*/test_gnn" --force --delete

-----
# Step 3:  Now train the model using the financial-fraud-training container


## Create training configuration file
NOTE: Training configuration file must conform to schema defined in docs (to be updated.)

__Important: Models and configuration files needed for deployment using NVIDIA Dynamo-Triton will be saved in model-repository under the folder that is mounted in /trained_models inside the container__

In [ ]:
import yaml

with open("../config/config.yaml", "r") as f:
    config = yaml.safe_load(f)

training_config = config["training"]

#### Save the training configuration as a json file

In [ ]:
training_config_file_name = 'training_config.json'

with open(os.path.join(training_config_file_name), 'w') as json_file:
    json.dump(training_config, json_file, indent=4)

# clone config to S3
! aws s3 cp ./training_config.json s3://$bucket_name/config/training_config.json

## Train model using financial_fraud_training container

In [ ]:
import sys
sys.path.insert(0, '..')  # Adjust path if needed to find src/

from src.model import SageMakerTrainingJob
from src.model.sagemaker_training import SageMakerTrainingConfig

# Create configuration
config = SageMakerTrainingConfig(
    bucket_name=bucket_name,
    sagemaker_training_role=sagemaker_training_role,
    training_repo=training_repo,
    # Optional overrides (defaults shown):
    # instance_type="ml.g5.xlarge",
    # cuda_compat_version="cuda-compat-13-0",
)

# Launch training job
training_job = SageMakerTrainingJob(config)
training_job_arn = training_job.launch()

# Access job name if needed
print(f"Job name: {training_job.training_job_name}")

#### Make sure that the training job succeeds
According to the training configuration file defined earlier, if the training runs successfully

In [ ]:
final_status = training_job.poll()

----
# Step 4:  Serve your python backend model using NVIDIA Dynamo-Triton
__!Important__: Change MODEL_REPO_PATH to point to `{model_output_dir}` / `python_backend_model_repository` if you used a different path in your training configuration file

#### Install NVIDIA Dynamo-Triton Client

In [ ]:
!pip install 'tritonclient[all]' --quiet

In [ ]:

from tritonclient.http import InferInput, InferRequestedOutput

In [ ]:
import tritonclient.grpc as triton_grpc
import tritonclient.http as httpclient

##### Replace HOST with the actual URL where your NVIDIA Dynamo-Triton server is hosted.


In [ ]:
HOST = inference_host
HTTP_PORT = 80
GRPC_PORT = 8006
METRICS_PORT = 8007

### Serve your models with NVIDIA Dynamo-Triton

With the infrastructure repo deployed, we have a Lambda function waiting for the training job to complete and for the models to be output to `s3://ml-on-containers-<accountnumber>/output` and then, they'll get extracted to a different bucket to be served by the inference host setup by our infrastructure.

### URLs for GRPC and HTTP request to the inference server

In [ ]:
client_grpc = triton_grpc.InferenceServerClient(url=f'{HOST}:{GRPC_PORT}')
client_http = httpclient.InferenceServerClient(url=f'{HOST}:{HTTP_PORT}')

#### Here’s an example of how to prepare data for inference, using random data

## Prediction without computing Shapley values

### Read preprocessed input transactions to send query to NVIDIA Dynamo-Triton

In [ ]:
from src.model.sagemaker_inference import prepare_and_send_inference_request, load_hetero_graph

In [ ]:
gnn_data_dir = os.path.join(data_root_dir, "gnn")
test_data = load_hetero_graph(gnn_data_dir)
compute_shap = False
result =  prepare_and_send_inference_request(test_data | {"COMPUTE_SHAP": np.array([compute_shap], dtype=np.bool_)})

In [ ]:
len(result['PREDICTION'])

#### Scores on test data

In [ ]:
from src.model.metrics import compute_score_for_batch

In [ ]:
# Decision threshold to flag a transaction as fraud
#Change to trade-off precision and recall
decision_threshold = 0.5
y = test_data['edge_label_user_to_merchant'].to_numpy(dtype=np.int32)
compute_score_for_batch(y, result['PREDICTION'], decision_threshold)

---
# Step 5: Latency and Throughput Tests

Measure inference performance with **realistic request sizes**:
- **Single transaction**: 1 edge + connected user/merchant (real-time fraud scoring)
- **Small batch**: 10-100 transactions (micro-batch processing)
- **Varying batch sizes**: Latency scaling analysis

### Create Realistic Test Samples
Generate subgraphs of varying sizes to simulate real-world request patterns.

In [ ]:
from src.utils.plotting import create_batch_samples, print_subgraph_stats

In [ ]:
# Load full test data
full_test_data = load_hetero_graph(gnn_data_dir)

# Create samples for different batch sizes
batch_sizes = [1, 10, 50, 100]
num_samples_per_size = 500  # Number of samples to create for each batch size

test_samples = {}
for batch_size in batch_sizes:
    samples = create_batch_samples(full_test_data, batch_size, num_samples_per_size, gnn_data_dir)
    test_samples[batch_size] = samples
    print_subgraph_stats(samples[0], f"Batch size {batch_size} (example)")

print(f"\nCreated {num_samples_per_size} samples for each batch size: {batch_sizes}")

### Single Transaction Latency (Real-time Fraud Scoring)
This is the most critical metric: how fast can we score a single incoming transaction?

In [ ]:
# Measure single transaction latency (batch_size=1)
print("Measuring SINGLE TRANSACTION latency (batch_size=1)...")
single_tx_latencies = measure_latency_for_samples(test_samples[1], compute_shap=False)
print_latency_stats(single_tx_latencies, "Single Transaction Latency (without SHAP)")

### Latency vs Batch Size
How does latency scale as we increase the number of transactions per request?

In [ ]:
from src.model.metrics import measure_latency_for_samples, print_latency_stats

In [ ]:
# Measure latency for different batch sizes
latency_by_batch_size = {}

for batch_size in batch_sizes:
    print(f"\nMeasuring latency for batch_size={batch_size}...")
    latencies = measure_latency_for_samples(test_samples[batch_size], compute_shap=False)
    latency_by_batch_size[batch_size] = latencies
    print_latency_stats(latencies, f"Batch Size = {batch_size} transactions")

### Throughput Test (Concurrent Single-Transaction Requests)
Simulate multiple users submitting transactions simultaneously.

In [ ]:
from src.model.metrics import measure_throughput

In [ ]:
# Test throughput with different concurrency levels using single-transaction requests
concurrency_levels = [1, 5, 10, 20]
throughput_results = {}

for num_workers in concurrency_levels:
    throughput, _ = measure_throughput(test_samples[1], num_workers=num_workers, compute_shap=False)
    throughput_results[num_workers] = throughput
    print()

### Visualizations

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 1. Latency vs Batch Size (box plot)
ax1 = axes[0]
batch_data = [latency_by_batch_size[bs] for bs in batch_sizes]
bp = ax1.boxplot(batch_data, labels=[str(bs) for bs in batch_sizes], patch_artist=True)
for patch in bp['boxes']:
    patch.set_facecolor('steelblue')
    patch.set_alpha(0.7)
ax1.set_xlabel('Batch Size (transactions)', fontsize=11)
ax1.set_ylabel('Latency (ms)', fontsize=11)
ax1.set_title('Latency vs Batch Size', fontsize=12)
ax1.grid(axis='y', alpha=0.3)

# 2. Throughput vs Concurrency
ax2 = axes[1]
workers = list(throughput_results.keys())
throughputs = list(throughput_results.values())
bars = ax2.bar(workers, throughputs, color='coral', edgecolor='black', alpha=0.8)
ax2.set_xlabel('Concurrent Workers', fontsize=11)
ax2.set_ylabel('Throughput (tx/second)', fontsize=11)
ax2.set_title('Throughput vs Concurrency\n(single-tx requests)', fontsize=12)
ax2.set_xticks(workers)
for bar, t in zip(bars, throughputs):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{t:.1f}', ha='center', fontsize=9)
ax2.grid(axis='y', alpha=0.3)

# 3. Single Transaction Latency Distribution
ax3 = axes[2]
ax3.hist(single_tx_latencies, bins=20, color='seagreen', edgecolor='black', alpha=0.7)
mean_lat = statistics.mean(single_tx_latencies)
p95_lat = sorted(single_tx_latencies)[int(len(single_tx_latencies) * 0.95)]
ax3.axvline(mean_lat, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_lat:.1f}ms')
ax3.axvline(p95_lat, color='orange', linestyle='--', linewidth=2, label=f'P95: {p95_lat:.1f}ms')
ax3.set_xlabel('Latency (ms)', fontsize=11)
ax3.set_ylabel('Frequency', fontsize=11)
ax3.set_title('Single Transaction Latency\nDistribution', fontsize=12)
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### Summary Table

In [ ]:
# Build summary table comparing latency across batch sizes
summary_rows = []
for batch_size in batch_sizes:
    lats = latency_by_batch_size[batch_size]
    lats_sorted = sorted(lats)
    n = len(lats)
    summary_rows.append({
        'Batch Size': batch_size,
        'Mean (ms)': f"{statistics.mean(lats):.2f}",
        'P50 (ms)': f"{lats_sorted[int(n * 0.50)]:.2f}",
        'P95 (ms)': f"{lats_sorted[int(n * 0.95)]:.2f}",
        'P99 (ms)': f"{lats_sorted[min(int(n * 0.99), n-1)]:.2f}",
        'Min (ms)': f"{min(lats):.2f}",
        'Max (ms)': f"{max(lats):.2f}",
        'Latency/Tx (ms)': f"{statistics.mean(lats) / batch_size:.2f}"
    })

summary_df = pd.DataFrame(summary_rows)
summary_df.set_index('Batch Size', inplace=True)

print("\n" + "="*70)
print("  LATENCY SUMMARY BY BATCH SIZE")
print("="*70)
display(summary_df)

# Key insights
single_tx_mean = statistics.mean(latency_by_batch_size[1])
batch_100_mean = statistics.mean(latency_by_batch_size[100])
print(f"\nKey Insights:")
print(f"  - Single transaction P95 latency: {sorted(latency_by_batch_size[1])[int(len(latency_by_batch_size[1]) * 0.95)]:.2f} ms")
print(f"  - Batching 100 tx reduces per-tx latency by {(1 - (batch_100_mean/100) / single_tx_mean) * 100:.1f}%")
print(f"  - Max throughput (20 workers): {throughput_results.get(20, 'N/A'):.1f} tx/s" if 20 in throughput_results else "")